In [ ]:
#                     RPM	    TPM	    RPD
# Gemini Embedding	  100	    30,000	1,000

# Solicitudes por minuto (RPM)
# Tokens por minuto (entrada) (TPM)
# Solicitudes por día (RPD)

Comprobación de que con canciones largas no de problema la generación de embeedings con gemini

In [1]:
from google import genai
from dotenv import load_dotenv
import os
load_dotenv() 
client = genai.Client(api_key=os.environ['GOOGLE_API_KEY'])
# client = genai.Client()

with open("cancion_large.txt", "r", encoding="utf-8") as f:
    cancion_mas_larga = f.read()

with open("cancion_large_in_chars.txt", "r", encoding="utf-8") as f2:
    cancion_large_in_chars = f2.read()

words = cancion_mas_larga.split(" ")
print(f"Longitud de la canción más larga: {len(words)} caracteres")

words2 = cancion_large_in_chars.split(" ")
print(f"Longitud de la canción más larga: {len(words2)} caracteres")
result = client.models.embed_content(
        model="gemini-embedding-001",
        contents= [
            "miau",
            cancion_mas_larga,
            cancion_large_in_chars
        ])

for embedding in result.embeddings:
    print(len(embedding.values), embedding.values)

Longitud de la canción más larga: 10124 caracteres
Longitud de la canción más larga: 10008 caracteres
3072 [-0.01307931, 0.00839118, 0.019608095, -0.102873854, -0.009463158, 0.01472984, -0.0060087866, -0.00096523215, 0.016759662, 0.013533837, -0.034180727, 0.006098773, -0.013628814, 0.004058341, 0.11741014, -0.018383155, -0.0032029015, 0.001065208, -0.018322423, -0.014000536, 0.008450755, -0.010374616, 0.015645813, -0.005938478, -0.0058148415, 0.020716682, 0.003296025, 0.0036768024, 0.02291456, -0.009611959, 0.0147500355, 0.0027086951, -0.006734099, -0.009009205, -0.017283002, 0.009610846, 0.010471226, 0.008495684, -0.016098898, -0.002235852, 0.008845762, 0.012594757, -0.017941182, 0.011304408, 0.0121252695, 0.011257332, 0.005532137, -0.034625072, 0.0031865253, -0.0039347736, 0.0043545384, 0.018300796, -0.0070240237, -0.14020994, 0.01914337, -0.012020982, -0.010985425, -0.00681856, 0.011357781, -0.0058900416, -0.03484051, -0.015768336, 0.013103323, -0.0021961255, -0.0065578143, -0.0147

No existe problema en la generación de embbedings con Gemini pero la cuota de trasferencias falta ajustarla para que no de errores

## Generación de Embbedings con gemini

In [2]:
import pandas as pd

In [3]:
# Carga del dataset

path_df = "../../../../data/spotify_dataset_sin_duplicados_4.csv"
df = pd.read_csv(path_df)
# column_to_process = df['text'].fillna("")


In [13]:
from google import genai
from dotenv import load_dotenv
import os
import time 
import numpy as np 

# Configuración de cuotas
RPM = 100         # Solicitudes por minuto
TPM = 30000       # Tokens por minuto
RPD = 1000        # Solicitudes por día



# Carga del archivo de enviroment
load_dotenv() 

# Configuración del api key de gemini 
client = genai.Client(api_key=os.environ['GOOGLE_API_KEY'])


# Variables para control de cuotas
requests_this_minute = 0
tokens_this_minute = 0
requests_today = 0
minute_start = time.time()
day_start = time.time()

embeddings = []
start = 0

counter_testing = 0
chunksize = 100
chunk_id = 0

END = False
# Para guardar
path_embbedings = "../../../../data/gemini_embeddings"
path_errors = "../../../../data/gemini_embeddings_errors"
os.makedirs(path_embbedings, exist_ok=True)
os.makedirs(path_errors, exist_ok=True)

for chunk in pd.read_csv(path_df, chunksize=chunksize):
    # Calcular tokens aproximados (1 token ≈ 4 caracteres en inglés, puede variar)
    texts = chunk['text'].fillna("")
    embeddings = []
    end = start - 1
    for idx, song in enumerate(texts, start=chunk_id*chunksize):
        print("Procesando la canción: ", idx)
        end = idx
        num_tokens = len(song) // 4
        
        # Control de solicitudes por día
        if requests_today >= RPD:
            print("Límite de solicitudes por día alcanzado.")
            print("Procederemos a guardar lo que tenemos")
            END = True
            embeddings.append(None)
            break

        # Control de tokens por minuto
        if tokens_this_minute + num_tokens > TPM:
            tiempo_restante = 60 - (time.time() - minute_start) + 20
            print(f"Límite de tokens por minuto alcanzado. Esperando {tiempo_restante:.2f} segundos...")
            time.sleep(max(0, tiempo_restante))
            tokens_this_minute = 0
            requests_this_minute = 0
            minute_start = time.time()


        # Control de solicitudes por minuto
        if requests_this_minute >= RPM:
            tiempo_restante = 60 - (time.time() - minute_start) + 20
            print(f"Límite de solicitudes por minuto alcanzado. Esperando {tiempo_restante:.2f} segundos...")
            time.sleep(max(0, tiempo_restante))
            tokens_this_minute = 0
            requests_this_minute = 0
            minute_start = time.time()

        # Solicitud de embedding
        try:
            result = client.models.embed_content(
                model="gemini-embedding-001",
                contents=[song]
            )
            embedding = result.embeddings[0].values
            embeddings.append(embedding)
            print("Cancion con id:", idx, "procesada")
        except Exception as e:
            print(f"Errro  en {idx}: {e}")
            print("->Descuidad lo intentaremos de nuevos en 2 minutos")
            # Esperar 1 minutico y volver a intentar sino sale pues ya guardar el None
            time.sleep(120)
            try:
                result = client.models.embed_content(
                    model="gemini-embedding-001",
                    contents=[song]
                )
                embedding = result.embeddings[0].values
                embeddings.append(embedding)
                print("Cancion con id:", idx, "procesada")
            except Exception as e:
                print(f"Error en la canción {idx} (reintento): {e}")
                embeddings.append(None)

        # Actualizar contadores
        requests_this_minute += 1
        tokens_this_minute += num_tokens
        requests_today += 1
    if END:
        break
    chunk_id+=1
    # Guardar en chunks por si sale error de code:
    
    # np.save(os.path.join(path_embbedings, f"embeddings{start}-{end}.npy"), embeddings)

    if embeddings:
        if any(e is None for e in embeddings):
            save_path = os.path.join(path_errors, f"embeddings_{start}-{end}.npy")
            print(f"Contiene None, guardando en carpeta de errores: {save_path}")
        else:
            # np.save(
            #     os.path.join(path_embbedings, f"embeddings_{start}-{end}.npy"),
            #     np.array(embeddings, dtype=object)
            # )
            save_path = os.path.join(path_embbedings, f"embeddings_{start}-{end}.npy")
            print(f" Guardando en carpeta normal: {save_path}")
        np.save(save_path, np.array(embeddings, dtype=object))
        # print(f"Guardado embeddings_{start}-{end}.npy con {len(embeddings)} filas")
    start = end + 1
    # chunk_id += 1
    # print(embeddings)
    print("Cantidad de embbedings guardados correctamente: ", len(embeddings))
    print("Dimension de los embbedings: ", len(embeddings[0]) if embeddings and embeddings[0] is not None else 0)

    print("!!!!! END:", end)




Procesando la canción:  0
Cancion con id: 0 procesada
Procesando la canción:  1
Cancion con id: 1 procesada
Procesando la canción:  2
Cancion con id: 2 procesada
Procesando la canción:  3
Cancion con id: 3 procesada
Procesando la canción:  4
Cancion con id: 4 procesada
Procesando la canción:  5
Cancion con id: 5 procesada
Procesando la canción:  6
Cancion con id: 6 procesada
Procesando la canción:  7
Cancion con id: 7 procesada
Procesando la canción:  8
Cancion con id: 8 procesada
Procesando la canción:  9
Cancion con id: 9 procesada
Procesando la canción:  10
Cancion con id: 10 procesada
Procesando la canción:  11
Cancion con id: 11 procesada
Procesando la canción:  12
Cancion con id: 12 procesada
Procesando la canción:  13
Cancion con id: 13 procesada
Procesando la canción:  14
Cancion con id: 14 procesada
Procesando la canción:  15
Cancion con id: 15 procesada
Procesando la canción:  16
Cancion con id: 16 procesada
Procesando la canción:  17
Cancion con id: 17 procesada
Procesando l

KeyboardInterrupt: 